In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from random import sample
import random
random.seed(14023799)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import statsmodels.formula.api as smf
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

### Boston Housing Data

In [2]:
dataset = load_boston()
boston_housing = pd.DataFrame(dataset.data, columns= map(str.lower, dataset.feature_names))
boston_housing['medv'] = dataset.target

### 2.2 Preparation
#### 2.2.1 Splitting data to training and testing samples

In [3]:
num = list(range(len(boston_housing)))
ran = sample(num, int(len(num)*0.8))
rem = [i for i in num if i not in ran] 

boston_train = boston_housing.iloc[ran, :]
boston_test = boston_housing.iloc[rem, :]

X_train = boston_train.iloc[:,:-1]
y_train = boston_train.iloc[:,-1:]

X_test = boston_test.iloc[:,:-1]
y_test = boston_test.iloc[:,-1:]

### 3.2 Best Subset Regression

In [4]:
def fit_linear_reg(X,Y):
    #Fit linear regression model and return RSS and R squared values
    model_k = LinearRegression(fit_intercept = True)
    model_k.fit(X,Y)
    RSS = mean_squared_error(Y, model_k.predict(X))*len(Y)
    R_squared = model_k.score(X,Y)
    return RSS, R_squared

#Implementing Best subset selection (using itertools.combinations)¶
#Importing tqdm for the progress bar
from tqdm import tnrange, tqdm_notebook
import itertools 

#Initialization variables
Y = y_train['medv']
X = X_train
k = len(y_train.columns) + 1
RSS_list, R_squared_list, feature_list = [],[],[]
numb_features = []

#Looping over k features in X
for k in tnrange(1,len(X.columns) + 1, desc = 'Loop...'):

    #Looping over all possible combinations: from 11 choose k
    for combo in itertools.combinations(X.columns,k):
        tmp_result = fit_linear_reg(X[list(combo)],Y)   #Store temp result 
        RSS_list.append(tmp_result[0])                  #Append lists
        R_squared_list.append(tmp_result[1])
        feature_list.append(combo)
        numb_features.append(len(combo))   

#Store in DataFrame
df_f = pd.DataFrame({'numb_features': numb_features,'RSS': RSS_list, 
                   'R_squared':R_squared_list,'features':feature_list})   

#Finding the best subsets for each number of features
#Using the smallest RSS value, or the largest R_squared value

df_min = df_f[df_f.groupby('numb_features')['RSS'].transform(min) == df_f['RSS']]
df_max = df_f[df_f.groupby('numb_features')['R_squared'].transform(max) == df_f['R_squared']]

Loop...:   0%|          | 0/13 [00:00<?, ?it/s]

In [7]:
df_max.iloc[9,3]

('crim', 'zn', 'nox', 'rm', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat')

### 3.3 Forward/Backward/Stepwise Regression Using AIC

#### 3.3.1 Backward Elimination

In [ ]:
def backward_elimination(data, target):
    
    best_features = data.columns.tolist()
    
    aic = sm.OLS(target, sm.add_constant(data[best_features])).fit().aic
    aic_values = [aic]

    while (True):
        new_aic = pd.Series(index = best_features)
        aic_diff = pd.Series(index = best_features)
        
        for new_column in best_features:
            model = sm.OLS(target, sm.add_constant(data[list(set(best_features) - set([new_column]))])).fit()
            new_aic[new_column] = model.aic
            aic_diff[new_column] = aic - model.aic
            
        max_aic = aic_diff.max()
        
        if(max_aic > 0):
            best_features.remove(aic_diff.idxmax())
            aic = new_aic[aic_diff.idxmax()]
            aic_values.append(aic)
            
        else:
            break
    
    final_AIC = sm.OLS(target, sm.add_constant(data[list(best_features)])).fit().aic
    
    return best_features, final_AIC

print('The variable coefficients provided by the model are:')
print(backward_elimination(X_train, y_train)[0])
print('\nFinal AIC value for the model is:')
print(backward_elimination(X_train, y_train)[1])

#### 3.3.2 Forward Selection

In [ ]:
def forward_selection(data, target):
    
    initial_features = data.columns.tolist()
    best_features = []
    
    aic = sm.OLS(target, sm.add_constant(data[best_features])).fit().aic
    aic_values = [aic]
    
    while (len(initial_features) > 0):
        remaining_features = list(set(initial_features) - set(best_features))
        new_aic = pd.Series(index = remaining_features)
        
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(data[best_features+[new_column]])).fit()
            new_aic[new_column] = model.aic
        min_aic = new_aic.min()
        
        if(min_aic < aic):
            aic = min_aic
            aic_values.append(aic)
            best_features.append(new_aic.idxmin())
        else:
            break
    
    final_AIC = sm.OLS(target, sm.add_constant(data[list(best_features)])).fit().aic
    
    return best_features, final_AIC

print('The variable coefficients provided by the model are:')
print(forward_selection(X_train, y_train)[0])
print('\nFinal AIC value for the model is:')
print(forward_selection(X_train, y_train)[1])

### Bi-directional elimination(Stepwise Selection)

In [ ]:
def stepwise_selection(data, target):
    
    initial_features = data.columns.tolist()
    best_features = []
    
    aic = sm.OLS(target, sm.add_constant(data[best_features])).fit().aic
    
    while (len(initial_features) > 0):
        remaining_features = list(set(initial_features) - set(best_features))
        forw_aic = pd.Series(index = remaining_features)
        
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(data[best_features+[new_column]])).fit()
            forw_aic[new_column] = model.aic
        min_aic = forw_aic.min()
        
        if(min_aic < aic):
            aic = min_aic
            best_features.append(forw_aic.idxmin())

            while (True):
                
                aic = sm.OLS(target, sm.add_constant(data[best_features])).fit().aic
                
                back_aic = pd.Series(index = best_features)
                aic_diff = pd.Series(index = best_features)

                for new_column in best_features:
                    model = sm.OLS(target, sm.add_constant(data[list(set(best_features) - set([new_column]))])).fit()
                    back_aic[new_column] = model.aic
                    aic_diff[new_column] = aic - model.aic

                max_aic = aic_diff.max()

                if(max_aic > 0):
                    best_features.remove(aic_diff.idxmax())
                    aic = back_aic[aic_diff.idxmax()]
                else:
                    break
                
        else:
            break
    
    final_AIC = sm.OLS(target, sm.add_constant(data[list(best_features)])).fit().aic
    return best_features, final_AIC

print('The variable coefficients provided by the model are:')
print(stepwise_selection(X_train, y_train)[0])

print('\nFinal AIC value for the model is:')
print(stepwise_selection(X_train, y_train)[1])